In [2]:
from time import time
from math import prod
import numpy as np
import matplotlib.pyplot as plt

# Density matrices and entanglement

## Obtaining the reduced density matrix of a bipartite pure system

Starting from a state of subsystem $A$ $|\psi_A\rangle$ and one of subsystem $B$ $|\psi_B\rangle$, compute:

$$
|\psi\rangle = |\psi_A\rangle \otimes |\psi_B\rangle
$$

<details>
  <summary>Solution part 1</summary>

```python
state = np.kron(s0, s1 )
```
</details>

In [3]:
num_sites = 2
local_dim0 = 2
local_dim1 = 2

s0 = [1, 0]
s1 = [0, 1]

state = ...

In [4]:
state = np.kron(s0,s1)

Compute the density matrix:

$$
\rho = |\psi\rangle\langle\psi|
$$

and afterwards compute the reduced density matrix of subsystem $A(B)$

<details>
  <summary>Solution part 2</summary>

```python
density_mat = np.outer(state, state.conj() )
```
</details>

<details>
  <summary>Solution part 3</summary>

```python
density_tens = density_mat.reshape([local_dim0, local_dim1, local_dim0, local_dim1])
...
if subsys_to_trace == 0:
    idxs = (0, 2)
else:
    idxs = (1, 3)
rho_A = np.trace(density_tens, axis1=idxs[0], axis2=idxs[1])
```
</details>

In [5]:
# Compute the density matrix using the outer product |\psi><\psi|
density_mat = np.outer(state, state.conj() )

# Reshape the 2-sites density matrix into a 4-legs tensor
#
#                     d0 -o- d0
# d0d1 -o- d0d1 ---->     o
#                     d1 -o- d1
#

subsys_to_trace = 0

print(density_mat)

density_tens = density_mat.reshape([local_dim0, local_dim1, local_dim0, local_dim1])

print(density_tens)

if subsys_to_trace == 0:
    idxs = (0, 2)
else:
    idxs = (1, 3)
rho_A = np.trace(density_tens, axis1=idxs[0], axis2=idxs[1])

# Trace away the subsystem you are interested in
#   _____
#   |   |
#   |_o_| d0
#     o
# d1 -o- d1
rho_A = np.trace(density_tens, axis1=idxs[0], axis2=idxs[1])

print(rho_A)

[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[[[0 0]
   [0 0]]

  [[0 1]
   [0 0]]]


 [[[0 0]
   [0 0]]

  [[0 0]
   [0 0]]]]
[[0 0]
 [0 1]]


## Computing the reduced density matrix without passing from the density matrix

We want to optimize the procedure to compute the reduced density matrix. There is
some additional computation that is not optimized at all.

Using `np.tensordot`, compute the reduced density matrix of subsystem $A(B)$

<details>
  <summary>Solution part 4</summary>

```python
if subsys_to_trace == 0:
    idxs = 0
else:
    idxs = 1

state = state.reshape([local_dim0, local_dim1])
rho_A = np.tensordot(state, state.conj(), ([idxs], [idxs]))
```
</details>

In [10]:
# Let's optimize a bit. We do not need the entire density matrix if we are interested only in reduced density matrices

if subsys_to_trace == 0:
    idxs = 0
else:
    idxs = 1

state = state.reshape([local_dim0, local_dim1])
rho_A = np.tensordot(state, state.conj(), ([idxs], [idxs]))

print(rho_A)

[[0 0]
 [0 1]]


Let's see if the optimization is real!

- Is the optimization the same is `local_dim0` is constant while `local_dim1` increases?
- Is the optimization the same is `local_dim1` is constant while `local_dim0` increases?

You might want to go with higher `local_dim` for these two tests, such that the product of the two is comparable
with the current numbers.

In [ ]:
# Is this really optimized?

3 = np.logspace(1,2.5,5, dtype=int)
local_dim1 = np.logspace(1,2.5,5,dtype=int)
timings = np.zeros((len(local_dim0), 2))

ii = 0
for d0, d1 in zip(local_dim0, local_dim1):
    print(d0,d1)
    # Put in first state
    s0 = np.zeros(d0)
    s0[0] = 1
    # Put in last state
    s1 = np.zeros(d1)
    s1[-1] = 1
    state = np.kron(s0, s1)

    tic = time()
    density_mat = np.outer(state, state.conj() )
    density_tens = density_mat.reshape([d0, d1, d0, d1])
    rho_A = np.trace(density_tens, axis1=1, axis2=3)
    timings[ii, 0] = time() - tic

    tic = time()
    state = state.reshape([d0, d1])
    rho_A = np.tensordot(state, state.conj(), ([1], [1]))
    timings[ii, 1] = time() - tic
    ii += 1


10 10
23 23
56 56
133 133


: 

In [1]:
fig, ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

ax[0].plot(local_dim1, timings[:, 0], "o--", label="Normal", color="forestgreen")
ax[0].plot(local_dim1, timings[:, 1], "s--", label="Tensordot", color="firebrick")
ax[0].legend(fontsize=14)
#ax[0].set_yscale("log")
ax[0].set_ylabel("Computational time [s]", fontsize=14)

ax[1].plot(local_dim1, timings[:, 0]/timings[:, 1], "o--", color="forestgreen")
ax[1].set_xlabel("Local dimension to trace away", fontsize=14)
ax[1].set_ylabel("Tensordot speedup", fontsize=14)
ax[0].set(xscale="log", yscale="log")
ax[1].set(xscale="log", yscale="log")

plt.show()

NameError: name 'plt' is not defined

## The general function for the reduced density matrix

Complete the function
<details>
  <summary>Solution part 5</summary>

```python
psi.reshape(*[loc_dim for _ in range(int(n_sites))])
```
</details>

In [6]:
def get_reduced_density_matrix(psi, loc_dim, n_sites, keep_indices,
    print_rho=False):
    """
    Parameters
    ----------
    psi : ndarray
        state of the Quantum Many-Body system
    loc_dim : int
        local dimension of each single site of the QMB system
    n_sites : int
        total number of sites in the QMB system
    keep_indices (list of ints):
        Indices of the lattice sites to keep.
    print_rho : bool, optional
        If True, it prints the obtained reduced density matrix], by default False

    Returns
    -------
    ndarray
        Reduced density matrix
    """
    if not isinstance(psi, np.ndarray):
        raise TypeError(f'density_mat should be an ndarray, not a {type(psi)}')

    if not np.isscalar(loc_dim) and not isinstance(loc_dim, int):
        raise TypeError(f'loc_dim must be an SCALAR & INTEGER, not a {type(loc_dim)}')

    if not np.isscalar(n_sites) and not isinstance(n_sites, int):
        raise TypeError(f'n_sites must be an SCALAR & INTEGER, not a {type(n_sites)}')

    # Ensure psi is reshaped into a tensor with one leg per lattice site
    psi_tensor = psi.reshape(*[loc_dim for _ in range(int(n_sites))])
    # Determine the environmental indices
    all_indices = list(range(n_sites))
    env_indices = [i for i in all_indices if i not in keep_indices]
    new_order = keep_indices + env_indices
    # Rearrange the tensor to group subsystem and environment indices
    psi_tensor = np.transpose(psi_tensor, axes=new_order)
    print(f"Reordered psi_tensor shape: {psi_tensor.shape}")
    # Determine the dimensions of the subsystem and environment for the bipartition
    subsystem_dim = np.prod([loc_dim for i in keep_indices])
    env_dim = np.prod([loc_dim for i in env_indices])
    # Reshape the reordered tensor to separate subsystem from environment
    psi_partitioned = psi_tensor.reshape((subsystem_dim, env_dim))
    # Compute the reduced density matrix by tracing out the env-indices
    RDM = np.tensordot(psi_partitioned, np.conjugate(psi_partitioned), axes=([1], [1]))
    # Reshape rho to ensure it is a square matrix corresponding to the subsystem
    RDM = RDM.reshape((subsystem_dim, subsystem_dim))

    # PRINT RHO
    if print_rho:
        print('----------------------------------------------------')
        print(f'DENSITY MATRIX TRACING SITES ({str(env_indices)})')
        print('----------------------------------------------------')
        print(RDM)

    return RDM